בס"ד

In [7]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import semopy

# -----------------------------
# 1. Generate mock data
# -----------------------------
np.random.seed(42)
n_samples = 300
n_diet = 700
n_microbes = 724

# Diet features (X)
diet = np.random.normal(size=(n_samples, n_diet))

# Microbes (M), partly driven by diet latent factor
true_diet_latent = diet.mean(axis=1)  # latent diet score
microbes = np.random.normal(size=(n_samples, n_microbes)) + true_diet_latent[:, None] * 0.3

# Triglycerides (Y), affected by both diet and microbes
triglycerides = (
    0.4 * true_diet_latent +
    0.8 * microbes.mean(axis=1) +
    np.random.normal(scale=1.0, size=n_samples)
)

# -----------------------------
# 2. PCA Reduction
# -----------------------------
# Diet block → 5 PCs
diet_scaled = StandardScaler().fit_transform(diet)
diet_pcs = PCA(n_components=5).fit_transform(diet_scaled)
diet_df = pd.DataFrame(diet_pcs, columns=[f"Diet_PC{i+1}" for i in range(5)])

# Microbiome block → 5 PCs
microbes_scaled = StandardScaler().fit_transform(microbes)
microbe_pcs = PCA(n_components=5).fit_transform(microbes_scaled)
microbe_df = pd.DataFrame(microbe_pcs, columns=[f"Micro_PC{i+1}" for i in range(5)])

# Combine into one DataFrame
df = pd.concat([diet_df, microbe_df], axis=1)
df["Triglycerides"] = triglycerides

# -----------------------------
# 3. SEM Model Definition
# -----------------------------
# Diet PCs → Micro PCs → Triglycerides
model_desc = """
# Regression paths
Micro_PC1 ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5
Micro_PC2 ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5
Micro_PC3 ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5
Micro_PC4 ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5
Micro_PC5 ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5

Triglycerides ~ Micro_PC1 + Micro_PC2 + Micro_PC3 + Micro_PC4 + Micro_PC5
Triglycerides ~ Diet_PC1 + Diet_PC2 + Diet_PC3 + Diet_PC4 + Diet_PC5
"""

# -----------------------------
# 4. Fit SEM
# -----------------------------
model = semopy.Model(model_desc)
res = model.fit(df)

# Get parameter estimates
estimates = model.inspect()
print(estimates.head())

# -----------------------------
# 5. Compute Indirect vs Direct
# -----------------------------
# Total indirect effect = sum(Diet_PC → Micro_PC × Micro_PC → Triglycerides)
diet_cols = [f"Diet_PC{i+1}" for i in range(5)]
micro_cols = [f"Micro_PC{i+1}" for i in range(5)]

direct_effects = estimates[(estimates["op"] == "~") &
                           (estimates["lhs"] == "Triglycerides") &
                           (estimates["rhs"].isin(diet_cols))]

indirect_effects = []
for d in diet_cols:
    total_indirect = 0
    for m in micro_cols:
        a = estimates.loc[(estimates["lhs"] == m) & (estimates["rhs"] == d), "Estimate"].values
        b = estimates.loc[(estimates["lhs"] == "Triglycerides") & (estimates["rhs"] == m), "Estimate"].values
        if len(a) > 0 and len(b) > 0:
            total_indirect += a[0] * b[0]
    indirect_effects.append({"Diet_PC": d, "Indirect": total_indirect})

indirect_df = pd.DataFrame(indirect_effects)
print("\nDirect Effects:")
print(direct_effects[["rhs", "Estimate"]])

print("\nIndirect Effects:")
print(indirect_df)


        lval op      rval  Estimate  Std. Err   z-value   p-value
0  Micro_PC1  ~  Diet_PC1 -0.017278  0.057200 -0.302059  0.762607
1  Micro_PC1  ~  Diet_PC2 -0.097088  0.058185 -1.668608  0.095195
2  Micro_PC1  ~  Diet_PC3 -0.014025  0.058339 -0.240406  0.810016
3  Micro_PC1  ~  Diet_PC4 -0.076690  0.058871 -1.302687  0.192682
4  Micro_PC1  ~  Diet_PC5  0.013455  0.059612  0.225717  0.821422


KeyError: 'lhs'